In [1]:
import atomictools as at
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='iframe'
from scipy.interpolate import RegularGridInterpolator, interp1d

In [2]:
def sph_to_cart(r, theta, phi):
    x = r*np.sin(theta)*np.cos(phi)
    y = r*np.sin(theta)*np.sin(phi)
    z = r*np.cos(theta)
    return x, y, z

def cart_to_sph(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z/r)
    phi = np.arctan(y/x)
    return r, theta, phi

In [3]:
def interpolate(phi_val, theta_val, Y):
    phi=np.linspace(0,2*np.pi, 100)
    theta=np.linspace(0, np.pi, 50)
    interp = RegularGridInterpolator((phi, theta), Y,
                                 bounds_error=False, fill_value=None, method="quintic")
    Y_val=interp((phi_val, theta_val))
    return Y_val

def interpolate_3d(r, r_dist, r_val, phi_val, theta_val, Y):
    Y_val = interpolate (phi_val, theta_val, Y)
    interp = interp1d(r, r_dist, fill_value="extrapolate")
    r_interp = interp(r_val)
    return r_interp**2 * np.abs(Y_val)**2

In [17]:
#utilizamos las funciones ya definidas en atomictools para crear objetos con los que se puede trabajar

class orbital():
    def __init__(self,f_rad, f_ang):
        
        l=f_rad.l
        m=f_ang.m
        Z=f_rad.Z
        
        
        self.R = f_rad
        rmax = self.R.rmax
        self.rmax = rmax
        self.r = self.R.r
        self.r_dist = self.R.P2.cumsum() * rmax / (self.R.npt-1)
        self.r_dist /= self.r_dist[-1]
        
        self.Y=f_ang
        self.theta = self.Y.theta
        
        if f_ang.switcher==True:
            self.theta_dist = (self.Y.ftheta_lm**2 * np.sin(self.theta)).cumsum() * np.pi/49.
            self.theta_dist /= self.theta_dist[-1]
        
        self.phi = self.Y.phi
        
        if f_ang.switcher==True:
            part=self.Y.part
            self.part=part
        
            if part is not None:
                self.phi_dist = (self.Y.fphi_m**2).cumsum() * 2. * np.pi / 99.
                self.phi_dist /= self.phi_dist[-1]
            
        self.x, self.y, self.z = np.mgrid[-rmax:rmax:40j, -rmax:rmax:40j, -rmax:rmax:40j]
        r, theta, phi = cart_to_sph(self.x, self.y, self.z)
        
        self.prob=interpolate_3d(self.r, self.R.R, r, phi, theta, self.Y.Y)

        
    def get_r(self, points=1):
            p = np.random.random(points)
            r = np.interp(p, self.r_dist, self.r)
            return r
    
    def get_theta(self, points=1):
            p = np.random.random(points)
            
            if f_ang.switcher == True:
                theta = np.interp(p, self.theta_dist, self.theta)
            elif f_ang.switcher == False:
                theta = np.interp(p, np.abs(self.theta_dist), self.theta)
            return theta
    
    def get_phi(self, points=1):
            if self.part is None:
                phi = np.random.random(points)*2.*np.pi
                return phi
            else:
                p = np.random.random(points)
                phi = np.interp(p, self.phi_dist, self.phi)
                return phi
        
    def plot_volume(self):
        
            min_val = np.min(self.prob)
            max_val = np.max(self.prob)
        
            fig = go.Figure(data=go.Volume(
               x=self.x.flatten(),
               y=self.y.flatten(),
               z=self.z.flatten(),
               value=self.prob.flatten(),
               opacity=0.1,
               isomin=0.0025*max_val,
               isomax=0.99*max_val,
               surface_count=100,
               colorscale='viridis'
               ))
            fig.show()
        
    def plot_scatter(self, points=100000, op=0.01):
            if points>6.7e5:
                print("The maximum number of points is 6.7e5.")
                points = 6.7e5
            points = int(points)
            r = self.get_r(points)
            theta = self.get_theta(points)
            phi = self.get_phi(points)
            rmax = self.rmax
        
            x, y, z = sph_to_cart(r, theta, phi)
            fig=go.Figure(data=go.Scatter3d(
                x=x,
                y=y,
                z=z,
                mode='markers',
                marker=dict(size=1., color = 'red'),
                opacity=op
                ))
            fig.update_layout(
               #showlegend=False,
               scene = dict(
                xaxis = dict(nticks=4, range=[-rmax, rmax]),
                yaxis = dict(nticks=4, range=[-rmax, rmax]),
                zaxis = dict(nticks=4, range=[-rmax, rmax]),
                aspectmode = 'cube'
               )
           )
            fig.show()


In [18]:
'''f_rad=at.R_hydrog(5,3)

f_ang=at.real_ang_function(3,0)
obj=orbital(f_rad, f_ang)
obj.plot_scatter()'''

'f_rad=at.R_hydrog(5,3)\n\nf_ang=at.real_ang_function(3,0)\nobj=orbital(f_rad, f_ang)\nobj.plot_scatter()'

In [19]:
'''f_rad=at.R_num("FeVII_2s.txt")
f_ang=at.real_ang_function(0,0, "Im")
obj=orbital(f_rad, f_ang)
obj.plot_volume()'''

'f_rad=at.R_num("FeVII_2s.txt")\nf_ang=at.real_ang_function(0,0, "Im")\nobj=orbital(f_rad, f_ang)\nobj.plot_volume()'

In [20]:
'''obj.plot_scatter()'''

'obj.plot_scatter()'

In [26]:
'''f_rad=at.R_hydrog(2,1)

functions=([1,0], [2,-1], [3,0], [4,-2])
coefs=(1,1,1,1)

f_ang=at.comb_ang_function(functions, coefs)

obj=orbital(f_rad, f_ang)
obj.plot_volume()'''

'f_rad=at.R_hydrog(2,1)\n\nfunctions=([1,0], [2,-1], [3,0], [4,-2])\ncoefs=(1,1,1,1)\n\nf_ang=at.comb_ang_function(functions, coefs)\n\nobj=orbital(f_rad, f_ang)\nobj.plot_volume()'

In [23]:
'''obj.plot_scatter()'''

'obj.plot_scatter()'

In [27]:
'''f_ang=at.spherical_harmonic(3,2)
f_ang.plot_phase()'''

'f_ang=at.spherical_harmonic(3,2)\nf_ang.plot_phase()'